In [1]:
import bokeh.models
import bokeh.plotting
import bokeh.io
import bokeh.layouts
import bokeh.palettes
import bokeh.transform
import bokeh.embed

from sp_project.data_preparation.prediction_preparation import *
from sp_project.data_preparation.db_client import get_global_db_client
from sp_project.notebook_tools import default_clients
import sp_project.app_state

bokeh.io.output_notebook()

/Users/missd/.pyenv/versions/3.10.4/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Loading BokehJS ...

### Get Data

In [2]:
lon = 7.70
lat = 46.42

In [3]:
async with default_clients() as (ow_client, db):
    app_state = sp_project.app_state.AppState(ow_client=ow_client, db_client=db)
    result = await extract_predictions_daily(app_state, lon=lon, lat=lat)

In [4]:
result

,temperature,wind_speed,rain,clouds
dt,,,,
2023-05-12 11:00:00+00:00,-3.56,1.98,0.00,89
2023-05-13 11:00:00+00:00,-2.83,2.18,0.00,90
2023-05-14 11:00:00+00:00,-2.31,2.32,0.00,93
2023-05-15 11:00:00+00:00,-1.77,2.68,0.00,92
2023-05-16 11:00:00+00:00,-5.61,2.65,0.00,100
2023-05-17 11:00:00+00:00,-5.24,1.53,0.00,100
2023-05-18 11:00:00+00:00,1.48,3.22,2.43,42
2023-05-19 11:00:00+00:00,-0.05,3.32,0.00,99


### Rain and average Temperature

In [5]:
day_source = bokeh.models.ColumnDataSource(data=result)

column_names = "avg_temp rain".split()

middle = result.index[len(result)//2]
selection_range = pd.Timedelta("2 days")

# Selected-Part-Plot
p = bokeh.plotting.figure(
    height=300, 
    width=800, 
    x_axis_type="datetime", 
    x_axis_location="above", 
    x_range=(middle-selection_range, middle+selection_range),
)

for tool in [bokeh.models.PanTool, bokeh.models.WheelZoomTool]:
    p.select_one(tool).dimensions = "width"


# Overview-Plot
overview = bokeh.plotting.figure(
    title="Drag the middle and edges of the selection box to change the range above",
    height=130, 
    width=800, 
    y_range=p.y_range,
    x_axis_type="datetime",
    y_axis_type=None,
    tools="hover", 
    toolbar_location=None, 
)

for fig in [p, overview]:
    fig.extra_y_ranges['rain'] = bokeh.models.Range1d(0, 5, bounds=(0, None))
    
    fig.vbar(
        x='dt',
        top="rain",
        source=day_source,    
        color="#0072B2",
        legend_label="Rain",
        width=pd.Timedelta(hours=20),
        y_range_name="rain",
    )

    fig.line(
        x='dt',
        y="temperature",
        source=day_source,    
        color="#D55E00",
        legend_label="Temperature",
    )

p.yaxis.axis_label = 'temperature [°C]'
p.yaxis.axis_label_text_color = "#D55E00"


rain_ax = bokeh.models.LinearAxis(y_range_name="rain", axis_label="rain [mm/h]")
rain_ax.axis_label_text_color ="#0072B2"
p.add_layout(rain_ax, 'right')
    
p.legend.location = "top_left"

range_tool = bokeh.models.RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

overview.ygrid.grid_line_color = None
overview.add_tools(range_tool)

line_fig = bokeh.layouts.column(p, overview)

bokeh.plotting.show(line_fig)

## Jupyter-Notebook Footer-Info

In [1]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 22.4.0
Datetime: 2023-05-14 16:21:08
Python Version: 3.10.4
-----------------------------------
